In [1]:
import sys
import nbimporter
from PyQt5.QtWidgets import QApplication, QMainWindow, QAction, QLabel, QFileDialog, QVBoxLayout, QWidget, QInputDialog
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import cv2
from match_faces import match_face, add_new_face

class AttendanceUI(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Face Recognition Attendance")
        self.setGeometry(100, 100, 600, 500)

        # Main widget
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        # Layout
        self.layout = QVBoxLayout()
        self.image_label = QLabel("No image loaded.")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.result_label = QLabel("Recognition Result: None")
        self.result_label.setAlignment(Qt.AlignCenter)

        self.layout.addWidget(self.image_label)
        self.layout.addWidget(self.result_label)
        self.central_widget.setLayout(self.layout)

        # Toolbar
        toolbar = self.addToolBar("Main Toolbar")

        load_action = QAction("Load Image", self)
        load_action.triggered.connect(self.load_and_recognize)
        toolbar.addAction(load_action)

        add_face_action = QAction("Add New Face", self)
        add_face_action.triggered.connect(self.add_new_face_ui)
        toolbar.addAction(add_face_action)

    def load_and_recognize(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg)")
        if file_name:
            self.load_image(file_name)
            matched_id, distance = match_face(file_name)
            if matched_id != "Unknown" and matched_id is not None:
                self.result_label.setText(f"Matched: {matched_id} (Distance: {distance:.3f})")
            else:
                self.result_label.setText("No match found.")

    def add_new_face_ui(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg)")
        if file_name:
            person_id, ok = QInputDialog.getText(self, "Add New Face", "Enter Person ID:")
            if ok and person_id:
                result = add_new_face(file_name, person_id)
                self.result_label.setText(result)

    def load_image(self, file_path):
        img_bgr = cv2.imread(file_path)
        if img_bgr is None:
            self.result_label.setText("Could not read image.")
            return

        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        height, width, channel = img_rgb.shape
        bytes_per_line = 3 * width
        q_img = QImage(img_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
        self.image_label.setPixmap(QPixmap.fromImage(q_img).scaled(300, 300, Qt.KeepAspectRatio))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    ui = AttendanceUI()
    ui.show()
    sys.exit(app.exec_())

C:\Users\25247\anaconda3\envs\facenet_env\lib\site-packages\facenet_pytorch\models\inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = tor

SystemExit: 0

C:\Users\25247\anaconda3\envs\facenet_env\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
